# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging

pd.set_option('max_colwidth',130)
pd.set_option('display.max_columns',None)
pd.set_option('display.precision',2)

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [ ]:
tracks = pd.read_parquet("tracks.parquet")
tracks

In [ ]:
catalog_names = pd.read_parquet("catalog_names.parquet")
catalog_names

In [ ]:
interactions = pd.read_parquet("interactions.parquet")
interactions

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [ ]:
for i in [tracks, catalog_names, interactions]:
    print(i.info(),'\nПропуски:\n',i.isna().sum(),'\n')

Пропусков нет, проверим явные дубликаты:

In [ ]:
for i in [catalog_names, interactions]:
    print(i.duplicated().sum(),'\n')

In [ ]:
tracks['track_id'].duplicated().sum()


Дублей так же нет. Таблица tracks содержит списки в ячейках, поэтому в ней проверили токлько track_id. Посмотрим распределения:

In [ ]:
for i in [catalog_names, interactions]:
    display(i.describe().style.format(precision=0,thousands=' '))

Посмотрим распределения track_seq на боксплоте, чтобы понять ситуацию:

In [ ]:
# Не тянет
# plt.figure(figsize=(15,6))
# sns.boxplot( interactions['track_seq'], color='#bbccbb' )
# plt.title( 'Плот', fontsize=20, pad=15 );


К сожалению, железо не справляется с 200 млн строк, но это и не проблема, тк у нас нет величин, как-либо связанных с распределениями - есть условные ID и даты, да и всё

# Выводы

Приведём выводы по первому знакомству с данными:
- явных проблем с данными нет,
- корерктирующий действий не понадобилось.

# === ЭТАП 2 ===

# EDA

Наиболее популярные треки, топ-30:

In [ ]:
interactions.groupby('track_id',as_index=False).agg({'user_id':'count'}).sort_values(by='user_id',ascending=False)[:30]

Наиболее популярные жанры, тоже топ-30:

In [ ]:
tracks.explode('genres').groupby('genres',as_index=False).agg({'artists':'count'}).sort_values(by='artists',ascending=False)[:30]

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

In [ ]:
interactions['started_at'] = pd.to_datetime(interactions['started_at'])  # Преобразование в формат datetime
# Выбор начальной даты (например, начало эпохи UNIX)
start_date = pd.Timestamp('1970-01-01')
# Преобразование дат в количество секунд с начальной даты
interactions['started_at'] = (interactions['started_at'] - start_date).dt.total_seconds() / (24 * 60 * 60)

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

In [ ]:
# import boto3
# s3 = boto3.client('s3')
# s3.upload_file('tracks.parquet', 's3-student-mle-20250529-05fed48463', 'items.parquet')
# s3.upload_file('interactions.parquet', 's3-student-mle-20250529-05fed48463', 'interactions.parquet')

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

In [ ]:
# Пример удаления переменных для освобождения оперативной памяти:
# del interactions, tracks

# === ЭТАП 3 ===

# Загрузка данных

"Если необходимо, то загружаем items.parquet, events.parquet" - нет, этого делать не нужно, ведь они у нас в оперативной памяти

In [ ]:
interactions['popularity'] = interactions.groupby('track_id')['user_id'].transform('count')
interactions


# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

In [ ]:
from sklearn.model_selection import train_test_split

# Допустим, у вас есть массив данных X и соответствующие целевые значения y
X_train, X_test, y_train, y_test = train_test_split(interactions.drop('popularity',axis=1), interactions['popularity'], test_size=0.2, random_state=42)

# Топ популярных

Рассчитаем рекомендации как топ популярных.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model = LinearRegression()

# Обучение модели на тренировочной выборке
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка качества модели
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.